In [2]:
!pip install selenium

In [3]:
!pip install msedge-selenium-tools selenium

In [1]:
import re
import csv
import datetime
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from msedge.selenium_tools import Edge, EdgeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element_by_xpath('.//span').text
    try:
        handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + responding
    
    tweet = (username, handle, postdate, text)
    return tweet

In [5]:
base = datetime.datetime(2020, 9, 18, 10, 15, 00, 00000)
date_list = [base - datetime.timedelta(days=x) for x in range(42)]

In [8]:
hoy = datetime.datetime.today()
hoy

datetime.datetime(2021, 3, 6, 10, 52, 12, 55855)

In [6]:
fechas = []
for date in date_list:
    fecha = str(date.year)+'-0'+str(date.month)+'-0'+str(date.day)
    fechas.append(fecha)

In [7]:
# create instance of web driver
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options)

# navigate to login screen
driver.get('https://twitter.com/login')
driver.maximize_window()
driver.implicitly_wait(15)
sleep(1)
data = []
tweet_ids = set()

for i in range(len(fechas)-1):
    fecha1 = fechas[i]
    fecha2 = fechas[i+1]
    driver.get('https://twitter.com/search?q=vacuna%20colombia%20-informe%20-noticia%20-reporte%20lang%3Aes%20until%3A'+fecha1+'%20since%3A'+fecha2+'%20-filter%3Alinks%20-filter%3Areplies&src=typed_query&f=live')
    # get all tweets on the page
    driver.implicitly_wait(15)
    last_position = driver.execute_script("return window.pageYOffset;")
    scrolling = True

    while scrolling:
        page_cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')
        for card in page_cards[-15:]:
            tweet = get_tweet_data(card)
            if tweet:
                tweet_id = ''.join(tweet)
                if tweet_id not in tweet_ids:
                    tweet_ids.add(tweet_id)
                    data.append(tweet)

        scroll_attempt = 0
        while True:
            # check scroll position
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(2)
            curr_position = driver.execute_script("return window.pageYOffset;")
            if last_position == curr_position:
                scroll_attempt += 1

                # end of scroll region
                if scroll_attempt >= 1:
                    scrolling = False
                    break
                else:
                    sleep(2) # attempt another scroll
            else:
                last_position = curr_position
                break


# close the web driver
driver.close()

KeyboardInterrupt: 

In [1]:
len(data)

NameError: name 'data' is not defined

In [152]:
with open('tweets_covid_'+str(len(data))+"-"+fechas[1]+'.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['Nombre de usuario', 'Usuario', 'Fecha', 'Texto']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)

# Extracción de tweets para el clasificador

## #YoMeVacuno

In [12]:
# create instance of web driver
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options)

# navigate to login screen
driver.get('https://twitter.com/login')
driver.maximize_window()
driver.implicitly_wait(15)
sleep(1)
data_positiva = []
tweet_ids_positiva = set()
driver.get('https://twitter.com/search?q=(%23YoMeVacuno)%20lang%3Aes%20-filter%3Alinks%20-filter%3Areplies&src=typed_query&f=live')
# get all tweets on the page
driver.implicitly_wait(15)
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True
for i in range(340):
    page_cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids_positiva:
                tweet_ids_positiva.add(tweet_id)
                data_positiva.append(tweet)
    # check scroll position
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    sleep(2)

# close the web driver
driver.close()

In [13]:
len(data_positiva)

5054

In [17]:
with open('tweets_covid_positivo2'+str(len(data_positiva))+'.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['Nombre de usuario', 'Usuario', 'Fecha', 'Texto']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data_positiva)

## #YoNoMeVacuno

In [20]:
# create instance of web driver
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options)

# navigate to login screen
driver.get('https://twitter.com/login')
driver.maximize_window()
driver.implicitly_wait(15)
sleep(1)
data_negativa = []
tweet_ids_negativa = set()
driver.get('https://twitter.com/search?f=live&q=(%23YoNoMeVacuno)%20lang%3Aes%20until%3A2020-11-01%20-filter%3Alinks%20-filter%3Areplies&src=typed_query')
# get all tweets on the page
driver.implicitly_wait(15)
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True
for i in range(160):
    print('Iteración: ',i,', tweets totales: ', len(data_negativa))
    page_cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids_negativa:
                tweet_ids_negativa.add(tweet_id)
                data_negativa.append(tweet)
    # check scroll position
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    sleep(2)

# close the web driver
driver.close()

Iteración:  0 , tweets totales:  0
Iteración:  1 , tweets totales:  15
Iteración:  2 , tweets totales:  30
Iteración:  3 , tweets totales:  30
Iteración:  4 , tweets totales:  30
Iteración:  5 , tweets totales:  41
Iteración:  6 , tweets totales:  56
Iteración:  7 , tweets totales:  56
Iteración:  8 , tweets totales:  56
Iteración:  9 , tweets totales:  56


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: MicrosoftEdge=89.0.774.45)


In [18]:
len(data_negativa)

3193

In [19]:
with open('tweets_covid_negativo2'+str(len(data_negativa))+'.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['Nombre de usuario', 'Usuario', 'Fecha', 'Texto']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data_negativa)

## Tweets para el generador de tweets

In [3]:
# create instance of web driver
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options)

# navigate to login screen
driver.get('https://twitter.com/login')
driver.maximize_window()
driver.implicitly_wait(15)
sleep(1)
data_gobierno = []
tweet_ids_gobierno = set()
driver.get('https://twitter.com/i/events/1242877452512145416')
# get all tweets on the page
driver.implicitly_wait(15)
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True
lista = []
i = 0
while scrolling:
    i = i+1
    print("Iteración: ", i, ". Número de tweets: ", len(data_gobierno))
    page_cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids_gobierno:
                tweet_ids_gobierno.add(tweet_id)
                data_gobierno.append(tweet)

    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1

            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(1) # attempt another scroll
        else:
            last_position = curr_position
            break

# close the web driver
driver.close()

Iteración:  1 . Número de tweets:  0
Iteración:  2 . Número de tweets:  4
Iteración:  3 . Número de tweets:  17
Iteración:  4 . Número de tweets:  29
Iteración:  5 . Número de tweets:  41
Iteración:  6 . Número de tweets:  53
Iteración:  7 . Número de tweets:  65
Iteración:  8 . Número de tweets:  78
Iteración:  9 . Número de tweets:  90
Iteración:  10 . Número de tweets:  101
Iteración:  11 . Número de tweets:  114
Iteración:  12 . Número de tweets:  126
Iteración:  13 . Número de tweets:  139
Iteración:  14 . Número de tweets:  153
Iteración:  15 . Número de tweets:  168
Iteración:  16 . Número de tweets:  183
Iteración:  17 . Número de tweets:  197
Iteración:  18 . Número de tweets:  212
Iteración:  19 . Número de tweets:  227
Iteración:  20 . Número de tweets:  242
Iteración:  21 . Número de tweets:  257
Iteración:  22 . Número de tweets:  270


AttributeError: 'NoneType' object has no attribute 'text'

In [28]:
len(data_gobierno)

258

In [29]:
data_gobierno[0]

('Nicolás García Bustos',
 '@nicolasgarciab',
 '2021-03-07T15:16:05.000Z',
 'En Albán nos reunimos con todos los alcaldes de la provincia del Gualivá para hablar de los avances del Plan de Vacunación y hacer seguimiento a los programas de Plan de Desarrollo que no podemos  descuidar durante la atención de la pandemia.\n#ATodaMarcha')

In [31]:
with open('tweets_covid_gobierno'+str(len(data_gobierno))+'.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['Nombre de usuario', 'Usuario', 'Fecha', 'Texto']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data_gobierno)

In [32]:
import pandas as pd

In [33]:
data1 = pd.read_csv('tweets_covid_gobierno183.csv')
data2 = pd.read_csv('tweets_covid_gobierno197.csv')
data3 = pd.read_csv('tweets_covid_gobierno258.csv')
data4 = pd.read_csv('tweets_covid_gobierno594.csv')
data5 = pd.read_csv('tweets_covid_gobierno685.csv')
data6 = pd.read_csv('tweets_covid_gobierno925.csv')
data7 = pd.read_csv('tweets_covid_gobierno989.csv')

In [34]:
tweets_gobierno = pd.concat([data1, data2, data3, data4, data5, data6, data7])
tweets_gobierno.dropna(inplace=True)
tweets_gobierno.drop_duplicates(inplace=True)
tweets_gobierno.reset_index(drop=True, inplace=True)
tweets_gobierno

,Nombre de usuario,Usuario,Fecha,Texto
0,Supersalud,@Supersalud,2021-03-06T20:30:15.000Z,@Supersalud\n Fabio Aristizábal visitó el depa...
1,MinSaludCol,@MinSaludCol,2021-03-06T20:30:00.000Z,#JuntosFrenteAlCovid Fueron muchos los hitos e...
2,MinSaludCol,@MinSaludCol,2021-03-06T18:31:28.000Z,"En la concha acústica de Mitú, Vaupés, el mini..."
3,SecretaríaSaludCesar,@CesarSecSalud,2021-03-06T03:32:46.000Z,#ReporteDiariio reportamos 35 nuevos casos de ...
4,Gobernación del Huila,@HuilaGob,2021-03-06T03:26:23.000Z,41 casos de Covid19 se confirmaron hoy para el...
...,...,...,...,...
3501,Presidencia Colombia,@infopresidencia,2020-11-07T00:32:33.000Z,“Hoy se registra el fallecimientos de 196 pers...
3502,Presidencia Colombia,@infopresidencia,2020-11-07T00:32:32.000Z,En #PrevenciónyAcción el Presidente \n@IvanDuq...
3503,Presidencia Colombia,@infopresidencia,2020-11-04T23:53:15.000Z,"""Del total de casos acumulados que se han regi..."
3504,Presidencia Colombia,@infopresidencia,2020-11-04T22:59:24.000Z,#EnVivo | Medidas de #PrevenciónyAcción imple...


In [35]:
fechas = tweets_gobierno.Fecha
for i in range(len(fechas)):
    fechas[i] = fechas[i][:10]

tweets_gobierno["Fecha"] = fechas

In [37]:
date_format = "%Y/%m/%d"
tweets_gobierno["Fecha"] = pd.to_datetime(tweets_gobierno["Fecha"], format=date_format)
tweets_gobierno

,Nombre de usuario,Usuario,Fecha,Texto
0,Supersalud,@Supersalud,2021-03-06,@Supersalud\n Fabio Aristizábal visitó el depa...
1,MinSaludCol,@MinSaludCol,2021-03-06,#JuntosFrenteAlCovid Fueron muchos los hitos e...
2,MinSaludCol,@MinSaludCol,2021-03-06,"En la concha acústica de Mitú, Vaupés, el mini..."
3,SecretaríaSaludCesar,@CesarSecSalud,2021-03-06,#ReporteDiariio reportamos 35 nuevos casos de ...
4,Gobernación del Huila,@HuilaGob,2021-03-06,41 casos de Covid19 se confirmaron hoy para el...
...,...,...,...,...
3501,Presidencia Colombia,@infopresidencia,2020-11-07,“Hoy se registra el fallecimientos de 196 pers...
3502,Presidencia Colombia,@infopresidencia,2020-11-07,En #PrevenciónyAcción el Presidente \n@IvanDuq...
3503,Presidencia Colombia,@infopresidencia,2020-11-04,"""Del total de casos acumulados que se han regi..."
3504,Presidencia Colombia,@infopresidencia,2020-11-04,#EnVivo | Medidas de #PrevenciónyAcción imple...


In [42]:
tweets_gobierno = tweets_gobierno.sort_values('Fecha', ascending = False)
tweets_gobierno

,Nombre de usuario,Usuario,Fecha,Texto
209,Supersalud,@Supersalud,2021-03-07,@Supersalud\n visitó el Hospital Regional de D...
234,Secretaría de Desarrollo de la Salud - Córdoba.,@CordobaSalud,2021-03-07,#BoletínCOVID19 Infórmate sobre los 5.795 vacu...
320,Gobernación de Sucre,@GobSucre,2021-03-07,"En Sucre se registran 60 nuevos casos, 53 recu..."
302,MinSaludCol,@MinSaludCol,2021-03-07,#JuntosFrenteAlCovid El \n@MinSaludCol\n puso ...
338,Alcaldía de Bogotá,@Bogota,2021-03-07,El Bosque El Abrazo es un espacio para conmemo...
...,...,...,...,...
981,SecretaríaSaludCesar,@CesarSecSalud,2020-03-09,"Hoy, \n@CesarSecSalud\n en conjunto con \n@Sec..."
982,SecretaríaSaludCesar,@CesarSecSalud,2020-03-09,Descarga CoronApp y despeja todas tus dudas so...
980,SecretaríaSaludCesar,@CesarSecSalud,2020-03-09,En la mesa sobre#Coronavirus #COVID19 particip...
983,SecretaríaSaludCesar,@CesarSecSalud,2020-03-07,Seguimos avanzando en acciones contra el #Deng...


In [44]:
tweets_gobierno.to_csv('tweets_gobierno.csv')

In [45]:
!pip install tweet-preprocessor

In [46]:
import preprocessor as p

In [51]:
def clean(s):
    p.set_options(p.OPT.EMOJI, p.OPT.URL, p.OPT.RESERVED)
    p.clean(s)
    s = s.replace("\n", "")
    return s

In [52]:
tweets_gobierno["Texto"] = tweets_gobierno["Texto"].apply(clean)
tweets_gobierno

,Nombre de usuario,Usuario,Fecha,Texto
209,Supersalud,@Supersalud,2021-03-07,@Supersalud visitó el Hospital Regional de Dui...
234,Secretaría de Desarrollo de la Salud - Córdoba.,@CordobaSalud,2021-03-07,#BoletínCOVID19 Infórmate sobre los 5.795 vacu...
320,Gobernación de Sucre,@GobSucre,2021-03-07,"En Sucre se registran 60 nuevos casos, 53 recu..."
302,MinSaludCol,@MinSaludCol,2021-03-07,#JuntosFrenteAlCovid El @MinSaludCol puso en m...
338,Alcaldía de Bogotá,@Bogota,2021-03-07,El Bosque El Abrazo es un espacio para conmemo...
...,...,...,...,...
981,SecretaríaSaludCesar,@CesarSecSalud,2020-03-09,"Hoy, @CesarSecSalud en conjunto con @SecSaludV..."
982,SecretaríaSaludCesar,@CesarSecSalud,2020-03-09,Descarga CoronApp y despeja todas tus dudas so...
980,SecretaríaSaludCesar,@CesarSecSalud,2020-03-09,En la mesa sobre#Coronavirus #COVID19 particip...
983,SecretaríaSaludCesar,@CesarSecSalud,2020-03-07,Seguimos avanzando en acciones contra el #Deng...


In [53]:
tweets_gobierno.to_csv('tweets_gobierno_clean.csv')